## Introduction
#### 데이터셋을 분류하고 각 그룹에 집계나 변형 같은 함수를 적용하는일은 데이터분석 과정의 핵심이다. 
#### pandas는 데이터 집합을 자연스럽게 나누고 요약할 수 있는 groupby라는 유연한 방법을 제공한다. 
#### 이 장에서 배우게 될 내용은 다음과 같다.
#### 1. 하나 이상의 키(함수, 배열, DataFrame의 컬럼이름)를 이용해서 객체를 여러 조각으로 나누는 방법
#### 2. 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
#### 3. 정규화, 선형회귀, 등급 또는 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법 
#### 4. 피벗 테이블과 교차 알람표를 구하는 방법
#### 5. 변위치 분석과 다른 통계 집단 분석을 수행하는 방법

## GroupBy 작동방법
#### 분리-적용-결합(split-apply-combine')은 그룹연산을 잘 설명하는 용어이다. 
#### 1. 그룹연산의 첫 번째는 데이터를 하나 이상의 키를 기준으로 분리한다. 예를들어 로우(axis=0) 또는 컬럼(axis=1)로 분리한다.
#### 2. 분리하고 나서 함수를 각 그룹에 적용시켜 새로운 값을 얻어낸다. 
#### 3. 함수를 적용한 결과를 하나의 객체로 결합한다. 
![ex_screenshot](그림10-1.png)

#### 각 그룹의 색인은 다음과 같이 다양한 형태가 될수 있으며, 모두 같은 타입일 필요가 없다 
####     1. 그룹으로 묶을 축과 동일한 길이의 리스트나 배열
####     2. DataFrame의 컬럼 이름을 지칭하는 값
####     3. 그룹으로 묶을 값과 그룹 이름에 대응하는 사전이나 Series 객체
####     4. 축 색인 혹은 색인 내의 개별 이름에 대해 실행되는 함수 
#### 1,2,3번의 세 방법은 객체를 나눌때 사용할 배열을 생성하기 위한 방법이다. 

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.131145,-0.605453
1,a,two,-0.395421,-0.342743
2,b,one,0.112348,-0.166057
3,b,two,-0.059159,-1.103825
4,a,one,1.634402,-0.054741


#### 위 데이터를 key1으로 묶고 각 그룹에서 data1의 평균을 구한다. 
#### 여러방법이 있지만, data1에 대해 groupby 메서드를 호출하고 key1 컬럼을 넘기는 방법을 사용한다. 

In [2]:
grouped = df['data1'].groupby(df['key1'])  #groubed 변수는 Groupby 객체이다.
grouped
#groubed 변수는 Groupby 객체이다.
#df['key1']에 의해 참조되는 중간값에 대한 것 외에는 아무것도 계산되지 않은 객체이다. 
#이 객체는 그룹 연산을 위해 필요한 모든 정보를 가지고 있으며, 각 그룹에 어떤 연산을 적용할 수 있게 해준다.

In [3]:
grouped.mean() #그룹별 평균을 구하려면 Groupby 객체의 mean 메서드 사용
# 데이터가 그룹색인에 따라 수집된다. 즉, key1 컬럼에 있는 유일한 값으로 색인되는 새로운 Series객체가 생성된다.
# 새로 생성된 객체의 색인은 'key1'인데, 그 이유는 DataFrame 컬럼인 df['key1']이기 때문이다.

key1
a    0.790042
b    0.026595
Name: data1, dtype: float64

#### 만일 여러개의 배열을 리스트로 넘겼다면 조금 다른 결과를 얻게 된다. 

In [4]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means  
#데이터를 두개의 색인으로 묶었고, 그 결과 계층적 색인을 가지는 Series 객체를 얻을 수 있다.

key1  key2
a     one     1.382774
      two    -0.395421
b     one     0.112348
      two    -0.059159
Name: data1, dtype: float64

In [5]:
means.unstack()

key2,one,two
key1,,
a,1.382774,-0.395421
b,0.112348,-0.059159


In [12]:
# 이 예제에서는 그룹의 색인 모두 Series 객체이다. 길이만 같다면 어떤 배열이라도 상관 없다. 
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.430406
            2006    0.460652
Ohio        2005    0.417554
            2006   -1.095420
Name: data1, dtype: float64

#### 한 그룹으로 묶을 정보는 주로 같은 DataFrame 안에서 찾게되는데, 이 경우 컬럼 이름(문자열, 숫자 혹은 다른 객체)을 넘겨서 그룹의 색인으로 사용할 수 있다. 

In [13]:
df

,key1,key2,data1,data2
0,a,one,0.663728,0.824136
1,a,two,0.430406,-0.959427
2,b,one,0.460652,0.664105
3,b,two,0.171381,1.039909
4,a,one,-1.095420,0.031450


In [14]:
df.groupby('key1').mean()
#key2 컬럼이 결과에서 빠졌다. df['key2']는 숫자 데이터가 아니다. 이런 컬럼은 성가신 컬럼(nuisance column)이라고 하며 결과에서 제외한다.

,data1,data2
key1,,
a,-0.000429,-0.034614
b,0.316016,0.852007


In [15]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.215846  0.427793
     two   0.430406 -0.959427
b    one   0.460652  0.664105
     two   0.171381  1.039909

In [16]:
df.groupby(['key1', 'key2']).size()
# size 메서드는 그룹의 크기를 담고 있는 Series를 반환한다.

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 그룹 간 순회하기
#### GroupBy 객체는 순회(iteration)을 지원하는데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환한다. 

In [17]:
df

,key1,key2,data1,data2
0,a,one,0.663728,0.824136
1,a,two,0.430406,-0.959427
2,b,one,0.460652,0.664105
3,b,two,0.171381,1.039909
4,a,one,-1.095420,0.031450


In [18]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.663728  0.824136
1    a  two  0.430406 -0.959427
4    a  one -1.095420  0.031450
b
  key1 key2     data1     data2
2    b  one  0.460652  0.664105
3    b  two  0.171381  1.039909


In [19]:
for (k1, k2), group in df.groupby(['key1', 'key2']):  #색인이 여럿 존재하는 경우 튜플의 첫 번째 원소가 색인값이 된다.
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.663728  0.824136
4    a  one -1.095420  0.031450
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.430406 -0.959427
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.460652  0.664105
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.171381  1.039909


#### 당연히 이 안에서 원하는 데이터만 골라낼 수 있다. 한줄이면 그룹별 데이터를 사전형으로 쉽게 바꾸어 사용가능하다. 

In [20]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one  0.663728  0.824136
 1    a  two  0.430406 -0.959427
 4    a  one -1.095420  0.031450,
 'b':   key1 key2     data1     data2
 2    b  one  0.460652  0.664105
 3    b  two  0.171381  1.039909}

In [21]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.460652,0.664105
3,b,two,0.171381,1.039909


In [22]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [23]:
#groupby메서드는 기본적으로 로우 기준(axis=0)으로 그룹을 만들지만, 다른축으로 만드는 것도 가능하다.
#df의 컬럼을 dtyte에 따라 그룹으로 묶을 수도 있다.
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.663728  0.824136
1  0.430406 -0.959427
2  0.460652  0.664105
3  0.171381  1.039909
4 -1.095420  0.031450
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 컬럼이나 컬럼의 일부만 선택하기 

In [24]:
'''
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
'''

"\ndf.groupby('key1')['data1']\ndf.groupby('key1')[['data2']]\n"

#### 위 코드는 아래 코드에에 대한 신택틱 슈가(같은 효과)로 같은 결과를 반환한다. 

In [24]:
'''
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
'''

"\ndf['data1'].groupby(df['key1'])\ndf[['data2']].groupby(df['key1'])\n"

#### 대용량 데이터를 다룰 경우 소수의 컬럼만 집계하고 싶을때가 종종 있다. 아래와 같이 작성한다. 

In [29]:
df.groupby(['key1', 'key2'])[['data2']].mean() #data2 컬럼에 대한 평균만 구하고 싶을때
#df.groupby(['key1', 'key2'])['data2'].mean() #data2 컬럼에 대한 평균만 구하고 싶을때

data2
key1 key2          
a    one   0.427793
     two  -0.959427
b    one   0.664105
     two   1.039909

In [31]:
s_grouped = df.groupby(['key1', 'key2'])['data2']  #groupby 메서드에 리스트나 배열을 넘겼을때는 Groupby객체가 된다.
s_grouped

In [32]:
s_grouped.mean()

key1  key2
a     one     0.427793
      two    -0.959427
b     one     0.664105
      two     1.039909
Name: data2, dtype: float64

### 사전과 Series에서 그룹핑 하기 

In [34]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns = ['a', 'b', 'c', 'd', 'e'],
                     index = ['Joe', 'steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan # a few NA values

people

,a,b,c,d,e
Joe,-0.775280,1.250849,-1.935024,1.218658,0.639837
steve,0.830856,-0.342407,0.478271,1.050025,-2.211058
Wes,-0.559445,NaN,NaN,1.468507,-1.332884
Jim,0.659787,0.255833,-0.440790,0.812515,0.352598
Travis,1.937332,-1.337738,0.393133,-0.922675,0.283741


In [35]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

In [39]:
by_column = people.groupby(mapping, axis=1)  #f는 사용되지 않음. 사용하지 않는 그룹 키도 문제 없다는것을 보여주는 예
by_column.sum()

#by_column = people.groupby(mapping, axis=1).sum()
#by_column

,blue,red
Joe,-0.716366,1.115407
steve,1.528296,-1.722609
Wes,1.468507,-1.892329
Jim,0.371725,1.268219
Travis,-0.529542,0.883335


In [40]:
map_series = pd.Series(mapping)

map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [41]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 함수로 그룹핑 하기 

In [42]:
people

,a,b,c,d,e
Joe,-0.775280,1.250849,-1.935024,1.218658,0.639837
steve,0.830856,-0.342407,0.478271,1.050025,-2.211058
Wes,-0.559445,NaN,NaN,1.468507,-1.332884
Jim,0.659787,0.255833,-0.440790,0.812515,0.352598
Travis,1.937332,-1.337738,0.393133,-0.922675,0.283741


In [49]:
people.groupby(len).sum() #배열을 넘기는 대신 len 함수를 넘긴 경우
# people은 사람의 이름을 색인값으로 사용하였다. len 함수를 넘기면 이름의 길이별로 그룹을 묶어준다.
#people.groupby(type).sum() #배열을 넘기는 대신 len 함수를 넘긴 경우

,a,b,c,d,e
3,-0.674937,1.506683,-2.375814,3.499681,-0.340448
5,0.830856,-0.342407,0.478271,1.050025,-2.211058
6,1.937332,-1.337738,0.393133,-0.922675,0.283741


In [51]:
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()  #내부적으로 모두 배열로 변환대므로 함수를 배열, 사전, Series와 섞어 쓰더러도 문제가 없다.

a         b         c         d         e
3 one -0.775280  1.250849 -1.935024  1.218658 -1.332884
  two  0.659787  0.255833 -0.440790  0.812515  0.352598
5 one  0.830856 -0.342407  0.478271  1.050025 -2.211058
6 two  1.937332 -1.337738  0.393133 -0.922675  0.283741

In [52]:
people

,a,b,c,d,e
Joe,-0.775280,1.250849,-1.935024,1.218658,0.639837
steve,0.830856,-0.342407,0.478271,1.050025,-2.211058
Wes,-0.559445,NaN,NaN,1.468507,-1.332884
Jim,0.659787,0.255833,-0.440790,0.812515,0.352598
Travis,1.937332,-1.337738,0.393133,-0.922675,0.283741


In [53]:
people.groupby([key_list]).sum()

,a,b,c,d,e
one,-0.503869,0.908442,-1.456753,3.737190,-2.904104
two,2.597119,-1.081904,-0.047657,-0.110161,0.636340


### 색인 단계로 그룹핑 하기 
#### 계층적으로 색인된 데이터는 축 색인의 단계 중 하나를 사용해서 편리하게 집계할 수 있다. 

In [54]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                   names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.721067  1.102624 -0.020828 -1.632355  0.928333
1     -1.524982  1.677817  0.712089  1.108141 -0.835444
2      0.386385 -0.362104 -0.193294  0.206593  0.999244
3     -0.361807 -0.433826 -1.000708 -0.651720  0.829892

In [55]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [56]:
hier_df.groupby(level='tenor', axis=1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


### 데이터 집계
#### 데이터 집계는 배열로부터 스칼라 값을 만들어 내는 모든 데이터 변환을 의미한다. 
#### count, sum, mean, median, std, var, min, max, prod, first, last
#### 직점 만들어낸 함수로 사용하는것도 가능하고, 객체에서 이미 정의된 메서드를 사용하는 것도 가능하다.
#### 예를들어 'quantile'메서드는 Series나 DataFrame 객체의 컬럼의 위치를 계산하는 메서드이다.

In [57]:
df

,key1,key2,data1,data2
0,a,one,0.663728,0.824136
1,a,two,0.430406,-0.959427
2,b,one,0.460652,0.664105
3,b,two,0.171381,1.039909
4,a,one,-1.095420,0.031450


In [61]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)
#quantile() 의 괄호 안에 0~1 사이의 값을 넣어주면 분위수를 계산해주며, 
#최소값과 최대값을 등분하여 그 사이를 interpolation 하여 분위수를 계산

key1
a    0.617064
b    0.431725
Name: data1, dtype: float64

#### 자신만의 데이터 집계함수를 사용하려면 배열의 aggregate나 agg 메서드에 해당 함수를 넘기면 된다 

In [63]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.759148,1.783563
b,0.289272,0.375804


In [64]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.000429  0.955439 -1.095420 -0.332507  0.430406  0.547067   
b      2.0  0.316016  0.204546  0.171381  0.243698  0.316016  0.388334   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.663728   3.0 -0.034614  0.893615 -0.959427 -0.463989  0.031450   
b     0.460652   2.0  0.852007  0.265733  0.664105  0.758056  0.852007   

                          
           75%       max  
key1                      
a     0.427793  0.824136  
b     0.945958  1.039909

### 컬럼에 여러가지 함수 적용하기 

In [65]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill'] #tip 데이터에 팀의 비율을 담기 위한 tip_pct를 추가

tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


#### 모든 컬럼을 집계하는것은 mean이나 std등의 메서드를 사용하거나, 원하는 함수에 aggregate를 사용하는 것이다.
#### 하지만, 컬럼에 따라 다른 함수를 사용하거나 여러개의 함수를 한꺼번에 적용하기 원한다면 아래의 예제로 가능하다. 

In [66]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [67]:
grouped_pct.agg(['mean', 'std', peak_to_peak])
#함수 목록이나 함수 이름을 넘기면 함수 이름을 컬럼으로 하는 DataFrame을 얻게 된다.

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

#### Groupby 객체에서 자동으로 지정하는 컬럼이름을 그대로 쓰지 않아도 된다. 
#### lambda : functions have the name "< lambda >" which is difficult to identify (you can check the function's __name__ attribute).
#### 이름과 함수가 담긴 튜플 리스트를 넘기면, 각 튜플의 첫번째 요소가 DataFrame의 컬럼이름으로사용된다.                                      

In [68]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])
#grouped_pct.agg([('foo', 'mean'), ('bar', 'std')])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

#### DataFrame은 컬럼마다 다른 함수를 적용하거나 여러개의 함수를 모든 컬럼에 적용할 수 있다. 
#### tip_pct와 total_bill 컬럼에 동일한 세가지 통계를 계산한다고 가정하자. 

In [69]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

<ipython-input-69-94f2b21cf8d2>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [70]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

#### 위의 예처럼 컬럼이름과 메서드가 담긴 튜플의 리스트를 넘기는 것도 가능하다. 

In [71]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

grouped['tip_pct', 'total_bill'].agg(ftuples)

<ipython-input-71-4dbd2ff45dc6>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [72]:
grouped.agg({'tip': np.max, 'size': 'sum'}) #컬럼마다 다른 함수를 적용하고 싶다면, 컬럼이름에 대응하는 함수가 들어있는 사전을 넘긴다.

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [73]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
            'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 색인되지 않은 형태로 집계된 데이터 변형하기 
#### as_index=False를 넘겨서 색인되지  않게 한다. 

In [75]:
#tips.groupby(['day', 'smoker']).mean()
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


### Apply: 일반적인 분리-적용-병합
#### 가장 일반적인 Groupby 메서드의 목적은 apply
#### apply메서드는 객체를 여러조각으로 나누고, 전달된 함수를 각 조각에 일괄 적용한 후 이를 다시 합친다. 
![ex_screenshot](./그림10-1.png)

In [76]:
tips[:5]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [77]:
tips.sort_values(by='tip_pct')

,total_bill,tip,smoker,day,time,size,tip_pct
237,32.83,1.17,Yes,Sat,Dinner,2,0.035638
102,44.30,2.50,Yes,Sat,Dinner,3,0.056433
57,26.41,1.50,No,Sat,Dinner,2,0.056797
0,16.99,1.01,No,Sun,Dinner,2,0.059447
187,30.46,2.00,Yes,Sun,Dinner,5,0.065660
...,...,...,...,...,...,...,...
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667


In [78]:
#팁 데이터에서 그룹별 상위 5개의 tip_pct 값을 고른다.
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]


top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [79]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

#### If you want to pass a function to apply with other arguments or keywords, you can pass them after the function.
#### NOTE - Beside basic usage, it requires creativity to use apply and get the most out of it. What happens inside a function depends on you, as long as the function returns a pandas object or scalar value.

In [82]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')
#tips.groupby(['smoker']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

#### Inside groupby, when you want to invoke a method, there is a shortcut for it.

In [83]:
# Normal route
result = tips.groupby('smoker')['tip_pct'].describe()

result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [84]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [85]:
grouped

In [86]:
# Shortcut

f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### 그룹 색인 생략하기
#### 앞의 예제에서 반환된 객체는 원본 객체의 각 조각에 대한 색인과 그룹키가 계층적으로 사용되었다.
#### 이런 결과는 groupby 메서드에 group_key=False를 넘겨서 막을 수 있다. 

In [87]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [88]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 변위치(Quantile) 및 버킷(Bucket) 분석
#### 8장에서 사용한 cut과 qcut 메서드를 groupby와 조합 사용

In [89]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                     'data2': np.random.randn(1000)})
frame

,data1,data2
0,1.597403,0.179896
1,-1.289893,-0.612043
2,-1.196593,-0.207222
3,-0.798372,1.942092
4,0.311237,-1.641908
...,...,...
995,1.372440,0.072321
996,-1.909049,-1.386564
997,0.400340,-0.132276
998,-0.451378,0.289471


In [90]:
quartiles = pd.cut(frame.data1, 4) #임의의 데이터 묶음을 cut을 사용하여 등간격 구간으로 나눔
quartiles[:10]

0      (1.269, 2.774]
1    (-1.739, -0.235]
2    (-1.739, -0.235]
3    (-1.739, -0.235]
4     (-0.235, 1.269]
5    (-1.739, -0.235]
6     (-0.235, 1.269]
7    (-3.249, -1.739]
8    (-1.739, -0.235]
9     (-0.235, 1.269]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.249, -1.739] < (-1.739, -0.235] < (-0.235, 1.269] < (1.269, 2.774]]

#### cut에서 반환된 categorical 객체는 바로 groupby로 넘길 수 있다.

In [95]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
           'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
#grouped.apply(get_stats)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.249, -1.739]",-2.191334,2.209129,41.0,0.153489
"(-1.739, -0.235]",-2.870219,4.230613,361.0,-0.024600
"(-0.235, 1.269]",-2.960643,2.849113,503.0,-0.020870
"(1.269, 2.774]",-3.033346,2.312964,95.0,-0.152275


In [96]:
grouping = pd.qcut(frame.data1, 10) #qcut을 사용하여 등간격이 아닌 크가가 같은 버킷을 계산
grouping

0                     (1.227, 2.774]
1      (-3.2439999999999998, -1.255]
2                   (-1.255, -0.806]
3                   (-0.806, -0.502]
4                     (0.233, 0.515]
                   ...              
995                   (1.227, 2.774]
996    (-3.2439999999999998, -1.255]
997                   (0.233, 0.515]
998                 (-0.502, -0.249]
999                   (0.806, 1.227]
Name: data1, Length: 1000, dtype: category
Categories (10, interval[float64]): [(-3.2439999999999998, -1.255] < (-1.255, -0.806] < (-0.806, -0.502] < (-0.502, -0.249] ... (0.233, 0.515] < (0.515, 0.806] < (0.806, 1.227] < (1.227, 2.774]]

In [97]:
grouping = pd.qcut(frame.data1, 10, labels=False) #qcut을 사용하여 등간격이 아닌 크가가 같은 버킷을 계산하고,
grouping                                          #label=False로 변위치 숫자를 구했다.

0      9
1      0
2      1
3      2
4      6
      ..
995    9
996    0
997    6
998    3
999    8
Name: data1, Length: 1000, dtype: int64

In [98]:
grouped = frame.data2.groupby(grouping)

grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.191334,2.209129,100.0,0.007462
1,-2.496900,2.565527,100.0,0.061789
2,-2.521541,1.942092,100.0,-0.170335
3,-2.870219,4.230613,100.0,0.079871
4,-2.960643,2.849113,100.0,0.097736
5,-2.485880,2.135456,100.0,-0.094460
6,-2.603700,2.163788,100.0,0.016327
7,-2.273566,2.541901,100.0,-0.187554
8,-1.839826,2.129268,100.0,0.075872


### 예제 : 그룹에 따른 값으로 결측치 채우기
#### 누락된 데이터를 정리할때 : dropna 사용
#### 하지만, 누락된 값을 고정된 값이나 혹은 데이터로부터 도출된 값으로 채우고 싶을때 : fillna 

In [2]:
import numpy as np
import pandas as pd

s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -2.397220
2         NaN
3    0.260182
4         NaN
5   -0.428990
dtype: float64

In [3]:
s.fillna(s.mean())

0   -0.855343
1   -2.397220
2   -0.855343
3    0.260182
4   -0.855343
5   -0.428990
dtype: float64

In [4]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
         'Oregon', 'Nevada', 'California', 'Idaho']

# Create 4 copies of elements in both East and West as key
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)

data

Ohio          0.872489
New York     -1.044069
Vermont      -0.186052
Florida      -0.042437
Oregon        0.116227
Nevada        0.347706
California    0.016218
Idaho        -1.458971
dtype: float64

In [5]:
# setting some values to NA
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.872489
New York     -1.044069
Vermont            NaN
Florida      -0.042437
Oregon        0.116227
Nevada             NaN
California    0.016218
Idaho              NaN
dtype: float64

In [6]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [7]:
data.groupby(group_key).mean()

East   -0.071339
West    0.066222
dtype: float64

In [8]:
# setting the NA values with group means
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.872489
New York     -1.044069
Vermont      -0.071339
Florida      -0.042437
Oregon        0.116227
Nevada        0.066222
California    0.016218
Idaho         0.066222
dtype: float64

#### 그룹에 따라 미리 정의된 다른 값을 채워넣어야 하는경우 : 각 그룹은 name속성을 가지고 있으므로 이를 이용 

In [9]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio          0.872489
New York     -1.044069
Vermont       0.500000
Florida      -0.042437
Oregon        0.116227
Nevada       -1.000000
California    0.016218
Idaho        -1.000000
dtype: float64

### 예제 : Random sampling and Permutation
#### 대용량의 데이터에서 랜덤 표본을 뽑아내고자 한다. 
#### Series의 sample 메서드를 사용 

In [10]:
suits = ['H', 'S', 'C', 'D'] #하트, 스페이드, 클럽, 다이아몬드
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
base_names

['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'K', 'Q']

In [11]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

In [12]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [14]:
def draw(deck, n=5):   #5장의 카드를 랜덤으로 뽑기위해 
    return deck.sample(n)

draw(deck)

5H    5
9D    9
7H    7
5C    5
2D    2
dtype: int64

In [17]:
cards

['AH',
 '2H',
 '3H',
 '4H',
 '5H',
 '6H',
 '7H',
 '8H',
 '9H',
 '10H',
 'JH',
 'KH',
 'QH',
 'AS',
 '2S',
 '3S',
 '4S',
 '5S',
 '6S',
 '7S',
 '8S',
 '9S',
 '10S',
 'JS',
 'KS',
 'QS',
 'AC',
 '2C',
 '3C',
 '4C',
 '5C',
 '6C',
 '7C',
 '8C',
 '9C',
 '10C',
 'JC',
 'KC',
 'QC',
 'AD',
 '2D',
 '3D',
 '4D',
 '5D',
 '6D',
 '7D',
 '8D',
 '9D',
 '10D',
 'JD',
 'KD',
 'QD']

In [22]:
# Get last letter for suit
get_suit = lambda card: card[-1]   #각 세트별로 2장을 무작위로 샘플링 한다. 세트는 각 카드이름의 마지막 글자이므로
                                   #이를 이용해 그룹을 나누고 apply사용  
deck.groupby(get_suit).apply(draw, n=2)

C  3C     3
   6C     6
D  KD    10
   9D     9
H  7H     7
   9H     9
S  AS     1
   3S     3
dtype: int64

In [23]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2) #색인하지 않고 각 그룹별로 무작위로 2장 추출

KC     10
4C      4
10D    10
QD     10
4H      4
5H      5
6S      6
10S    10
dtype: int64

### 예제: 그룹가중평균(Group Weighted Average)과 상관(Correlation)

In [24]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                               'b', 'b', 'b', 'b'],
                  'data': np.random.randn(8),
                  'weights': np.random.randn(8)})
df

,category,data,weights
0,a,0.944267,-1.075040
1,a,0.680431,0.936013
2,a,-0.009895,-1.570929
3,a,-0.031574,-0.633689
4,b,0.535661,-2.133437
5,b,-0.341282,-1.460552
6,b,1.186368,0.265003
7,b,0.500638,-1.657322


In [25]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights']) #category별 가중 편균

grouped.apply(get_wavg)

category
a    0.146217
b    0.232570
dtype: float64

#### 좀 더 복잡한 예제로 '야후 파이낸스'에서 가져온 주식과 S&P 500(종목코드 SPX) 지수의 종가 데이터를 샇펴본다. 

In [26]:
close_px = pd.read_csv('stock_px_2.csv', parse_dates = True,
                      index_col = 0)

close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [27]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


#### 퍼센트 변화율로 일일 수익률을 계산하여 연간 SPX 지수와의 상관을 살펴본다. 

In [28]:
close_px[0:3]

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01


In [29]:
spx_corr = lambda x: x.corrwith(x['SPX']) #SPX컬럼과 다른 컬럼과의 상관관계를 계산하는 함수
rets = close_px.pct_change().dropna() #close_px의 퍼센트 변화율을 계산한다. 
rets

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


In [30]:
spx_corr = lambda x: x.corrwith(x['SPX']) #SPX컬럼과 다른 컬럼과의 상관관계를 계산하는 함수
rets = close_px.pct_change().dropna() #close_px의 퍼센트 변화율을 계산한다. 
get_year = lambda x: x.year    #각 datatime에서 연도 속성만 반환하는 함수

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [31]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT'])) #개별 두 컬럼나의 상관을 계산하는 것도 가능

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: 그룹별 선형회귀 
#### groupby can also be used to perform complex group-wise statistical analysis, as long as the function returns pandas object or scalar value.
#### eg - We can perform Ordinary Least Square (OLS) regression of chunks of data.

In [32]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


### 피벗 테이블과 교차일람표
#### 'pivot table'은 데이터 요약화 도구이다. 
####  'pivot table'은 데이터를 하나이상의 키로 수집해서 어떤 키는 로우에, 어떤 키는 컬럼에 나열해서 데이터를 정렬한다. 

In [35]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill'] #tip 데이터에 팀의 비율을 담기 위한 tip_pct를 추가
#tips
tips.pivot_table(index=['day', 'smoker']) #day와 smoker집단의 평균(pivot_table의 기본 연산)

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

#### pivot_table allows you to aggregate by certain columns, group by certaing columns with 'index' parameters, and define columns with 'columns' parameter.
#### We could augment this table to include partial totals by passing 'margins=True'. It will add All row and column labels with corresponding values being group statistics for all data within single tier.
#### The 'All' values are aggregations without taking into account the present groups in the columns or rows.

In [36]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [37]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

#### To use a different aggregation function, pass it to 'aggfunc'. eg - 'count' or len will give cross-tabulation (frequencies) of group sizes.
#### If some combinations are empty (NA), you can pass a 'fill_value'.

In [38]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', 
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [39]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', 
                 aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulations: Crosstab(교차일람표)
#### 교차일람표는 그룹 빈도를 계산하기 위한 피벗 테이블의 특수한 경우이다. 

In [40]:
d = {0: [1, 'USA', 'Right'],
    1: [2, 'JP', 'Left'],
    2: [3, 'USA', 'Left'],
    3: [4, 'JP', 'Right'],
    4: [5, 'JP', 'Left'],
    5: [6, 'USA', 'Left']}
data = pd.DataFrame.from_dict(d, orient='index',
                              columns=['Sample', 'Nationality', 'Handedness'])
data

,Sample,Nationality,Handedness
0,1,USA,Right
1,2,JP,Left
2,3,USA,Left
3,4,JP,Right
4,5,JP,Left
5,6,USA,Left


In [41]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left,Right,All
Nationality,,,
JP,2,1,3
USA,2,1,3
All,4,2,6


In [42]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244